In [4]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [5]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [6]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA_MECVV = tabla[
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) |
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0)]   
SIN_PA = tabla[
          (tabla["P_diagnosticada"] == 1.0) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30)]

# Síndromes geriàtrics

## Index de Charslton

## Caigudes prèvies

In [7]:
caprev_CON_PA= (CON_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_CON_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [8]:
caprev_CON_PA_MECVV= (CON_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_CON_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [9]:
caprev_SIN_PA= (SIN_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SIN_PA:.2f}%")

Percentatge de caigudes prèvies: 6.12%


#### Pvalor

##### Normalitat

In [10]:
stat, p_value = shapiro(CON_PA['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [11]:
stat, p_value = shapiro(CON_PA_MECVV['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [12]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [13]:
stat, p_value_12 = mannwhitneyu(CON_PA['caidas_previas'], CON_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.49002535950179893


In [14]:
stat, p_value_13 = mannwhitneyu(CON_PA['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [15]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.44143809205435813


## Deliris

In [16]:
del_CON_PA= (CON_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_CON_PA:.2f}%")

Percentatge de deliris: 23.53%


In [17]:
del_CON_PA_MECVV= (CON_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_CON_PA_MECVV:.2f}%")

Percentatge de deliris: 40.00%


In [18]:
del_SIN_PA= (SIN_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SIN_PA:.2f}%")

Percentatge de deliris: 18.37%


#### Pvalor

##### Normalitat

In [19]:
stat, p_value = shapiro(CON_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [20]:
stat, p_value = shapiro(CON_PA_MECVV['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [21]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [22]:
stat, p_value_12 = mannwhitneyu(CON_PA['delirios'], CON_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.39174655228054167


In [23]:
stat, p_value_13 = mannwhitneyu(CON_PA['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.6549000779898075


In [24]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.13960802912048415


## Demència

In [25]:
dem_CON_PA= (CON_PA['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_CON_PA:.2f}%")

KeyError: 'demència'

In [ ]:
dem_CON_PA_MECVV= (CON_PA_MECVV['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_CON_PA_MECVV:.2f}%")

In [ ]:
dem_SIN_PA= (SIN_PA['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SIN_PA:.2f}%")

#### Pvalor

##### Normalitat

In [ ]:
stat, p_value = shapiro(CON_PA['demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

In [ ]:
stat, p_value = shapiro(CON_PA_MECVV['demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

In [26]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [27]:
stat, p_value_12 = mannwhitneyu(CON_PA['demència'], CON_PA_MECVV['demència'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

KeyError: 'demència'

In [28]:
stat, p_value_13 = mannwhitneyu(CON_PA['demència'], SIN_PA['demència'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

KeyError: 'demència'

In [29]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['demència'], SIN_PA['demència'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

KeyError: 'demència'

## Síndrome depressiu

In [30]:
depre_CON_PA= (CON_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_CON_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [31]:
depre_CON_PA_MECVV= (CON_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_CON_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 0.00%


In [32]:
depre_SIN_PA= (SIN_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SIN_PA:.2f}%")

Percentatge de síndrome depressiu: 14.29%


#### Pvalor

##### Normalitat

In [33]:
stat, p_value = shapiro(CON_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [34]:
stat, p_value = shapiro(CON_PA_MECVV['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [35]:
stat, p_value = shapiro(SIN_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [36]:
stat, p_value_12 = mannwhitneyu(CON_PA['sindrome_depresivo'], CON_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.11180527770715322


In [37]:
stat, p_value_13 = mannwhitneyu(CON_PA['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.3880726918610017


In [38]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.21340049774345338


## Incontinència urinària

In [39]:
ic_CON_PA= (CON_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_CON_PA:.2f}%")

Percentatge de incontinència urinària: 29.41%


In [40]:
ic_CON_PA_MECVV= (CON_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_CON_PA_MECVV:.2f}%")

Percentatge de incontinència urinària: 30.00%


In [41]:
ic_SIN_PA= (SIN_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_SIN_PA:.2f}%")

Percentatge de incontinència urinària: 14.29%


#### Pvalor

##### Normalitat

In [42]:
stat, p_value = shapiro(CON_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [43]:
stat, p_value = shapiro(CON_PA_MECVV['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5942
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [44]:
stat, p_value = shapiro(SIN_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [45]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_uri'], CON_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [46]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.17015925929193942


In [47]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.237514563226218


## Incontinència fecal 

In [48]:
ifec_CON_PA= (CON_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_CON_PA:.2f}%")

Percentatge de incontinència fecal: 29.41%


In [49]:
ifec_CON_PA_MECVV= (CON_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_CON_PA_MECVV:.2f}%")

Percentatge de incontinència fecal: 10.00%


In [50]:
ifec_SIN_PA= (SIN_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_SIN_PA:.2f}%")

Percentatge de incontinència fecal: 12.24%


#### Pvalor

##### Normalitat

In [51]:
stat, p_value = shapiro(CON_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [52]:
stat, p_value = shapiro(CON_PA_MECVV['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [53]:
stat, p_value = shapiro(SIN_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3844
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [54]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_fec'], CON_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.2649031747093591


In [55]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.10681676423525503


In [56]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.8568943909879498


## Úlceres de pressió

In [57]:
up_CON_PA= (CON_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_CON_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [58]:
up_CON_PA_MECVV= (CON_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_CON_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [59]:
up_SIN_PA= (SIN_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SIN_PA:.2f}%")

Percentatge de úlceres de pressió: 2.04%


#### Pvalor

##### Normalitat

In [60]:
stat, p_value = shapiro(CON_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [61]:
stat, p_value = shapiro(CON_PA_MECVV['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [62]:
stat, p_value = shapiro(SIN_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [63]:
stat, p_value_12 = mannwhitneyu(CON_PA['ulceras_presion'], CON_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.49002535950179893


In [64]:
stat, p_value_13 = mannwhitneyu(CON_PA['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.4440308448923743


In [65]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6843175494727258


## Immobilitat

## Confusió

## Osteoporosi

In [66]:
ost_CON_PA= (CON_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_CON_PA:.2f}%")

Percentatge de osteoporosi: 0.00%


In [67]:
ost_CON_PA_MECVV= (CON_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_CON_PA_MECVV:.2f}%")

Percentatge de osteoporosi: 0.00%


In [68]:
ost_SIN_PA= (SIN_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_SIN_PA:.2f}%")

Percentatge de osteoporosi: 0.00%


#### Pvalor

##### Normalitat

In [69]:
stat, p_value = shapiro(CON_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [70]:
stat, p_value_12 = mannwhitneyu(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [71]:
stat, p_value_12 = ttest_ind(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV nan


## Sarcopènia

In [72]:
sarc_CON_PA= (CON_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_CON_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [73]:
sarc_CON_PA_MECVV= (CON_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_CON_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 10.00%


In [74]:
sarc_SIN_PA= (SIN_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SIN_PA:.2f}%")

Percentatge de sarcopènia: 4.08%


#### Pvalor

##### Normalitat

In [75]:
stat, p_value = shapiro(CON_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [76]:
stat, p_value = shapiro(CON_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [77]:
stat, p_value = shapiro(SIN_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [78]:
stat, p_value_12 = mannwhitneyu(CON_PA['sarcopenia'], CON_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.21976847950216105


In [79]:
stat, p_value_13 = mannwhitneyu(CON_PA['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.41519527301502746


In [80]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.45734646700671056


## Trastorns del son

In [81]:
ts_CON_PA= (CON_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_CON_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [82]:
ts_CON_PA_MECVV= (CON_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_CON_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [83]:
ts_SIN_PA= (SIN_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SIN_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


## Dolor crònic

In [84]:
dc_CON_PA= (CON_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_CON_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [85]:
dc_CON_PA_MECVV= (CON_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_CON_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [86]:
dc_SIN_PA= (SIN_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SIN_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


## Iatrogènic

In [87]:
iatr_CON_PA= (CON_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_CON_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [88]:
iatr_CON_PA_MECVV= (CON_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_CON_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [89]:
iatr_SIN_PA= (SIN_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SIN_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


## Restrenyiment

In [90]:
est_CON_PA= (CON_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_CON_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [91]:
est_CON_PA_MECVV= (CON_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_CON_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [92]:
est_SIN_PA= (SIN_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SIN_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


# Historial mèdic/comorbilitats

## Malalties CV

In [93]:
cv_CON_PA= (CON_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_CON_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [94]:
cv_CON_PA_MECVV= (CON_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_CON_PA_MECVV:.2f}%")

Percentatge de problemes CV: 10.00%


In [95]:
cv_SIN_PA= (SIN_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SIN_PA:.2f}%")

Percentatge de problemes CV: 6.12%


#### Pvalor

##### Normalitat 

In [96]:
stat, p_value = shapiro(CON_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [97]:
stat, p_value = shapiro(CON_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [98]:
stat, p_value = shapiro(SIN_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [99]:
stat, p_value_12 = mannwhitneyu(CON_PA['CV'], CON_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.7400102360583588


In [100]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [101]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6762982127217047


## Malalties del cor

In [102]:
cora_CON_PA= (CON_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_CON_PA:.2f}%")

Percentatge de malalties de cor: 11.76%


In [103]:
cora_CON_PA_MECVV= (CON_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_CON_PA_MECVV:.2f}%")

Percentatge de malalties de cor: 10.00%


In [104]:
cora_SIN_PA= (SIN_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SIN_PA:.2f}%")

Percentatge de malalties de cor: 14.29%


#### Pvalor

##### Normalitat

In [105]:
stat, p_value = shapiro(CON_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [106]:
stat, p_value = shapiro(CON_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [107]:
stat, p_value = shapiro(SIN_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [108]:
stat, p_value_12 = mannwhitneyu(CON_PA['probl_corazon'], CON_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.9265568396984611


In [109]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [110]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6762982127217047


## Malalties neurodegeneratives

In [111]:
nd_CON_PA= (CON_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_CON_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 0.00%


In [112]:
nd_CON_PA_MECVV= (CON_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_CON_PA_MECVV:.2f}%")

Percentatge de malalties neurodegeneratives: 10.00%


In [113]:
nd_SIN_PA= (SIN_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SIN_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 4.08%


#### Pvalor

##### Normalitat

In [114]:
stat, p_value = shapiro(CON_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [115]:
stat, p_value = shapiro(CON_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [116]:
stat, p_value = shapiro(SIN_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [117]:
stat, p_value_12 = mannwhitneyu(CON_PA['neurodegenerativas'], CON_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.21976847950216105


In [118]:
stat, p_value_13 = mannwhitneyu(CON_PA['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.41519527301502746


In [119]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.45734646700671056


## Diabetes mellitus

In [120]:
dm_CON_PA= (CON_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_CON_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [121]:
dm_CON_PA_MECVV= (CON_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_CON_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 10.00%


In [122]:
dm_SIN_PA= (SIN_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SIN_PA:.2f}%")

Percentatge de diabetes mellitus: 26.53%


#### Pvalor 

##### Normalitat

In [123]:
stat, p_value = shapiro(CON_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [124]:
stat, p_value = shapiro(CON_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [125]:
stat, p_value = shapiro(SIN_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [126]:
stat, p_value_12 = mannwhitneyu(CON_PA['DM'], CON_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.4120280546315189


In [127]:
stat, p_value_13 = mannwhitneyu(CON_PA['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PAN", p_value_13)

P-valor entre CON_PA i SIN_PAN 0.8163159339942008


In [128]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PAN", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PAN 0.2728301710651724


## Hepatopaties

In [129]:
hp_CON_PA= (CON_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_CON_PA:.2f}%")

Percentatge de hepatopaties: 11.76%


In [130]:
hp_CON_PA_MECVV= (CON_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_CON_PA_MECVV:.2f}%")

Percentatge de hepatopaties: 10.00%


In [131]:
hp_SIN_PA= (SIN_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SIN_PA:.2f}%")

Percentatge de hepatopaties: 6.12%


#### Pvalor

##### Normalitat

In [132]:
stat, p_value = shapiro(CON_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [133]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [134]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [135]:
stat, p_value_12 = mannwhitneyu(CON_PA['hepatopatias'], CON_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.9265568396984611


In [136]:
stat, p_value_13 = mannwhitneyu(CON_PA['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.4618706673466527


In [137]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6762982127217047


## Neoplàsia

In [138]:
neopl_CON_PA= (CON_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_CON_PA:.2f}%")

Percentatge de neoplàsia: 0.00%


In [139]:
neopl_CON_PA_MECVV= (CON_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_CON_PA_MECVV:.2f}%")

Percentatge de neoplàsia: 0.00%


In [140]:
neopl_SIN_PA= (SIN_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SIN_PA:.2f}%")

Percentatge de neoplàsia: 6.12%


#### Pvalor

##### Normalitat

In [141]:
stat, p_value = shapiro(CON_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [142]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [143]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [144]:
stat, p_value_12 = mannwhitneyu(CON_PA['neoplasias'], CON_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [145]:
stat, p_value_13 = mannwhitneyu(CON_PA['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_12)

P-valor entre CON_PA i SIN_PA 1.0


In [146]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.44143809205435813


## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [147]:
CON_PA.loc[:, 'creatinina'] = pd.to_numeric(CON_PA['creatinina'], errors='coerce')
CON_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(CON_PA_MECVV['creatinina'], errors='coerce')
SIN_PA.loc[:, 'creatinina'] = pd.to_numeric(SIN_PA['creatinina'], errors='coerce')

In [148]:
ecr_CON_PA= (CON_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_CON_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [149]:
ecr_CON_PA_MECVV= (CON_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_CON_PA_MECVV:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [150]:
ecr_SIN_PA= (SIN_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SIN_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 6.12%


#### Pvalor

##### Normalitat

In [151]:
stat, p_value = shapiro(CON_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9549
Valor p = 0.7439
El nombre de files segueix una distribució normal


In [152]:
stat, p_value = shapiro(CON_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9815
Valor p = 0.9110
El nombre de files segueix una distribució normal


In [153]:
stat, p_value = shapiro(SIN_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7235
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [154]:
stat, p_value_12 = ttest_ind(CON_PA['creatinina'], CON_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV nan


In [155]:
stat, p_value_13 = mannwhitneyu(CON_PA['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA nan


In [156]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA nan


## Fallida renal aguda (o insuficiència renal aguda)

In [157]:
ira_CON_PA= (CON_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_CON_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [158]:
ira_CON_PA_MECVV= (CON_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_CON_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [159]:
ira_SIN_PA= (SIN_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SIN_PA:.2f}%")

Percentatge de fallida renal aguda: 26.53%


#### Pvalor

##### Normalitat

In [160]:
stat, p_value = shapiro(CON_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [161]:
stat, p_value = shapiro(CON_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6553
Valor p = 0.0003
El nombre de files no segueix una distribució normal.


In [162]:
stat, p_value = shapiro(SIN_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [163]:
stat, p_value_12 = mannwhitneyu(CON_PA['ARF'], CON_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.17639454522781728


In [164]:
stat, p_value_13 = mannwhitneyu(CON_PA['ARF'], SIN_PA['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.8163159339942008


In [165]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ARF'], SIN_PA['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_23)

P-valor entre CON_PA i SIN_PA 0.14880872192595962


## Marejos/síncope/lipotímia/reacció vasovagal

In [166]:
dizsyn_CON_PA= (CON_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_CON_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [167]:
dizsyn_CON_PA_MECVV= (CON_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_CON_PA_MECVV:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [168]:
dizsyn_SIN_PA= (SIN_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SIN_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 2.04%


#### Pvalor

##### Normalitat

In [169]:
stat, p_value = shapiro(CON_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [170]:
stat, p_value = shapiro(CON_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [171]:
stat, p_value = shapiro(SIN_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [172]:
stat, p_value_12 = ttest_ind(CON_PA['mareos'], CON_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV nan


In [173]:
stat, p_value_12 = mannwhitneyu(CON_PA['mareos'], CON_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [174]:
stat, p_value_13 = mannwhitneyu(CON_PA['mareos'], SIN_PA['mareos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.5793276078637376


In [175]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['mareos'], SIN_PA['mareos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6843175494727258


## VIH

In [176]:
vih_CON_PA= (CON_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_CON_PA:.2f}%")

Percentatge de VIH: 0.00%


In [177]:
vih_CON_PA_MECVV= (CON_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_CON_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [178]:
vih_SIN_PA= (SIN_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SIN_PA:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [179]:
stat, p_value = shapiro(CON_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [180]:
stat, p_value = shapiro(CON_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [181]:
stat, p_value = shapiro(SIN_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


## Psicosi

In [182]:
psi_CON_PA= (CON_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_CON_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [183]:
psi_CON_PA_MECVV= (CON_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_CON_PA_MECVV:.2f}%")

Percentatge de psicosi: 0.00%


In [184]:
psi_SIN_PA= (SIN_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SIN_PA:.2f}%")

Percentatge de psicosi: 0.00%


## Deficiències nutricionals

In [185]:
nutri_CON_PA= (CON_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_CON_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [186]:
nutri_CON_PA_MECVV= (CON_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_CON_PA_MECVV:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [187]:
nutri_SIN_PA= (SIN_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SIN_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%
